<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/BinaryVariable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/WideTable.xlsx')
df = df.iloc[:, 1:]
df.head()

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L14:WBC,Sex,Age,NATNL,Location,PricipalProblemCode,PricipalProblemName,InPatient,OutPatient,Emergency
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,9.41,หญิง,75.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,NaN,หญิง,76.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,5.42,หญิง,78.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,NaN,หญิง,79.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,F411,Generalized anxiety disorder,0.0,1.0,0.0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,NaN,หญิง,58.0,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,NaN,NaN,0.0,1.0,0.0


In [55]:
data = pd.DataFrame(df)
remove = ['L12:Hemoglobin','L13:Hematocrit','L16:TIBC','L17:Tranferin SAT',
          'L18:Ferritin','L25:Parathyroid hormone','L26:Vitamin D','L27:UrineAlbumin/Cr ratio',
          'L28:UrineProtein/Cr ratio','L29:UrineVolume (24hrs)','L30:UrineUrea (24hrs)',
          'L31:UrineSodium (24hrs)','L32:UrineProtein (24hrs)']
data = data.drop(remove, axis=1)

In [56]:
data['Age'] = data['Age'].fillna('69')
data['Sex'] = data['Sex'].fillna('หญิง')
data['NATNL'] = data['NATNL'].fillna('ไทย')
data['Location'] = data['Location'].fillna('ห้องปฏิบัติการจุลทรรศน์วินิจฉัย')
data['InPatient'] = data['InPatient'].fillna(0)
data['OutPatient'] = data['OutPatient'].fillna(1)
data['Emergency'] = data['Emergency'].fillna(0)
data['PricipalProblemCode'] = data['PricipalProblemCode'].fillna('None')
data['PricipalProblemName'] = data['PricipalProblemName'].fillna('None')

In [57]:
null_count = data.isnull().sum()
null_columns = null_count[null_count > 0]
non_null_columns = null_count[null_count == 0]
print("Columns with Non-Null Values:")
for column_name in non_null_columns.index:
    print(column_name)
print("\nColumns with Null Values and Their Counts:")
print(null_columns)

Columns with Non-Null Values:
PatientUID
PatientVisitUID
Slopes
G1M2:ARB
G3M1:Statin
G5M2:NSAIDs
G1M5:Diuretic
G1M7:Beta blocker
G1M3:Dihydropyridine CCB
G1M6:MRA
G1M1:ACEI
G2M2:Metformin
G2M3:DPP4 inhibitor
G2M1:SU
G3M3:Fibrate
G2M5:SGLT2i
G4M1:Allopurinol
G1M9:Vasodilator
G1M8:Alpha blocker
G2M6:Insulin
G2M4:TZD
G4M2:Febuxostat
G3M2:Ezetimibe
G1M4:Non-dihydropyridine CCB
G5M1:Ketoanalog amino acid
G2M7:GLP1-RA
G4M3:Uricosuric agent
D1:Hypertension
D2:Diabetes millitus
D3:Dyslipidemia
D4:SLE
D5:Systemic sclerosis
D6:Rheumatoid arthritis
D7:HIV infection
D8:HBV infection
D9:HCV infection
D10:Ischemic heart disease
D11:Heart failure
L1:eGFR
Sex
Age
NATNL
Location
PricipalProblemCode
PricipalProblemName
InPatient
OutPatient
Emergency

Columns with Null Values and Their Counts:
Systolic BP             4884
Diastolic BP            4913
BMI                    14807
BSA                    14808
L2:BUN                 13507
L3:Creatinine              1
L4:Sodium              20808
L5:Potassiu

**Add One Binary Variable**


```
* Mode = If the data type is 'object'.
* Mean = If the data type is numeric (not 'object') and the missing percentage is less than 10%.
* Median = If the data type is numeric (not 'object') and the missing percentage is 10% or higher.

```



In [58]:
# Columns to impute using MissForest #22
columns_to_impute = ['Systolic BP', 'Diastolic BP', 'BMI', 'BSA', 'L5:Potassium',
                     'L7:Bicarbonate', 'L8:Uric', 'L9:Calcium', 'L10:Phosphorus',
                     'L11:Albumin', 'L14:WBC', 'L15:Platelet', 'L19:Blood sugar',
                     'L20:HbA1c', 'L22:Triglyceride', 'L24:HDL Cholesterol',
                     'L2:BUN','L3:Creatinine','L4:Sodium','L6:Chloride',
                     'L21:Cholesterol','L23:LDL Cholesterol']
data[columns_to_impute]

,Systolic BP,Diastolic BP,BMI,BSA,L5:Potassium,L7:Bicarbonate,L8:Uric,L9:Calcium,L10:Phosphorus,L11:Albumin,...,L19:Blood sugar,L20:HbA1c,L22:Triglyceride,L24:HDL Cholesterol,L2:BUN,L3:Creatinine,L4:Sodium,L6:Chloride,L21:Cholesterol,L23:LDL Cholesterol
0,138,76,20.72,1.38,3.8,29.7,NaN,NaN,NaN,NaN,...,100,5.3,62,65,6.5,0.82,143,103,193,129.0
1,134,66,20.27,1.36,4.3,NaN,NaN,NaN,NaN,NaN,...,89,NaN,69,76,NaN,0.88,NaN,NaN,205,139.0
2,103,57,19.05,1.33,4.4,27.2,NaN,NaN,NaN,NaN,...,88,5.5,74,60,14.5,1.10,141,102,198,129.0
3,128,62,NaN,NaN,4.2,28.2,NaN,NaN,NaN,NaN,...,93,5.3,71,68,15.4,0.72,131,96,179,100.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111,6.2,97,51,20.8,1.08,NaN,NaN,143,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,147,87,NaN,NaN,4.6,27.4,NaN,NaN,NaN,NaN,...,135,6.6,146,48,NaN,2.04,137,99,181,98.0
52615,143,86,NaN,NaN,5.2,23.5,NaN,9.2,4.0,4.2,...,133,7.2,104,47,28.3,1.87,141,104,125,62.0
52616,131,79,NaN,NaN,4.4,19.4,10.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15.5,1.09,141,110,NaN,NaN
52617,138,NaN,NaN,NaN,4.5,22.6,7.3,NaN,NaN,3.2,...,NaN,NaN,NaN,NaN,40.8,2.14,126,90,121,NaN


In [59]:
# Function to add binary indicator column for missing values
def add_missing_indicator(data, column):
    missing_indicator = data[column].isnull().astype(int)
    indicator_column_name = f'{column}_missing'
    data[indicator_column_name] = missing_indicator

# Function to automatically select imputation method and return it
def automatic_imputation_selection(data, column):
    dtype = data[column].dtype
    missing_percentage = data[column].isnull().mean() * 100

    if dtype == 'object':
        imputation_method = 'mode'
    elif missing_percentage < 10:
        imputation_method = 'mean'
    else:
        imputation_method = 'median'

    return imputation_method

# Sort the columns alphabetically
columns_to_impute.sort()

# Create a dictionary to store the imputation method for each column
imputation_methods = {}

# Split the data into four DataFrames
# 1. Original DataFrame
original_data = data.copy()

# 2. DataFrame with binary indicators
data_with_binary = data.copy()
for column in columns_to_impute:
    add_missing_indicator(data_with_binary, column)

# 3. DataFrame with automatic imputation (median, mode, mean) + binary indicators
data_with_imputation = data.copy()
for column in columns_to_impute:
    imputation_method = automatic_imputation_selection(data_with_imputation, column)
    imputation_methods[column] = imputation_method
    if imputation_method == 'mean':
        data_with_imputation[column].fillna(data_with_imputation[column].mean(), inplace=True)
    elif imputation_method == 'median':
        data_with_imputation[column].fillna(data_with_imputation[column].median(), inplace=True)
    elif imputation_method == 'mode':
        data_with_imputation[column].fillna(data_with_imputation[column].mode().iloc[0], inplace=True)
    add_missing_indicator(data_with_imputation, column)

# 4. DataFrame with only automatic imputation (median, mode, mean)
data_with_only_imputation = data.copy()
for column in columns_to_impute:
    imputation_method = automatic_imputation_selection(data_with_only_imputation, column)
    imputation_methods[column] = imputation_method
    if imputation_method == 'mean':
        data_with_only_imputation[column].fillna(data_with_only_imputation[column].mean(), inplace=True)
    elif imputation_method == 'median':
        data_with_only_imputation[column].fillna(data_with_only_imputation[column].median(), inplace=True)
    elif imputation_method == 'mode':
        data_with_only_imputation[column].fillna(data_with_only_imputation[column].mode().iloc[0], inplace=True)

# Print the imputation methods for each column
print("\nImputation Methods:")
for column, method in imputation_methods.items():
    print(f"{column}: {method}")



Imputation Methods:
BMI: mode
BSA: mode
Diastolic BP: mode
L10:Phosphorus: mode
L11:Albumin: mode
L14:WBC: mode
L15:Platelet: mode
L19:Blood sugar: mode
L20:HbA1c: mode
L21:Cholesterol: mode
L22:Triglyceride: mode
L23:LDL Cholesterol: median
L24:HDL Cholesterol: mode
L2:BUN: median
L3:Creatinine: mean
L4:Sodium: mode
L5:Potassium: mode
L6:Chloride: mode
L7:Bicarbonate: mode
L8:Uric: median
L9:Calcium: mode
Systolic BP: mode


In [67]:
original_data[columns_to_impute]

,BMI,BSA,Diastolic BP,L10:Phosphorus,L11:Albumin,L14:WBC,L15:Platelet,L19:Blood sugar,L20:HbA1c,L21:Cholesterol,...,L24:HDL Cholesterol,L2:BUN,L3:Creatinine,L4:Sodium,L5:Potassium,L6:Chloride,L7:Bicarbonate,L8:Uric,L9:Calcium,Systolic BP
0,20.72,1.38,76,NaN,NaN,9.41,313,100,5.3,193,...,65,6.5,0.82,143,3.8,103,29.7,NaN,NaN,138
1,20.27,1.36,66,NaN,NaN,NaN,NaN,89,NaN,205,...,76,NaN,0.88,NaN,4.3,NaN,NaN,NaN,NaN,134
2,19.05,1.33,57,NaN,NaN,5.42,434,88,5.5,198,...,60,14.5,1.10,141,4.4,102,27.2,NaN,NaN,103
3,NaN,NaN,62,NaN,NaN,NaN,NaN,93,5.3,179,...,68,15.4,0.72,131,4.2,96,28.2,NaN,NaN,128
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111,6.2,143,...,51,20.8,1.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,NaN,NaN,87,NaN,NaN,NaN,NaN,135,6.6,181,...,48,NaN,2.04,137,4.6,99,27.4,NaN,NaN,147
52615,NaN,NaN,86,4.0,4.2,NaN,NaN,133,7.2,125,...,47,28.3,1.87,141,5.2,104,23.5,NaN,9.2,143
52616,NaN,NaN,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.5,1.09,141,4.4,110,19.4,10.9,NaN,131
52617,NaN,NaN,NaN,NaN,3.2,18.70,298,NaN,NaN,121,...,NaN,40.8,2.14,126,4.5,90,22.6,7.3,NaN,138


In [69]:
# 2. DataFrame with binary indicators
data_with_binary

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L24:HDL Cholesterol_missing,L2:BUN_missing,L3:Creatinine_missing,L4:Sodium_missing,L5:Potassium_missing,L6:Chloride_missing,L7:Bicarbonate_missing,L8:Uric_missing,L9:Calcium_missing,Systolic BP_missing
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,0,1,0,1,0,1,1,1,1,0
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [71]:
# 3. DataFrame with automatic imputation (median, mode, mean) + binary indicators
data_with_imputation

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L24:HDL Cholesterol_missing,L2:BUN_missing,L3:Creatinine_missing,L4:Sodium_missing,L5:Potassium_missing,L6:Chloride_missing,L7:Bicarbonate_missing,L8:Uric_missing,L9:Calcium_missing,Systolic BP_missing
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
# 4. DataFrame with only automatic imputation (median, mode, mean)
data_with_only_imputation[columns_to_impute]

,BMI,BSA,Diastolic BP,L10:Phosphorus,L11:Albumin,L14:WBC,L15:Platelet,L19:Blood sugar,L20:HbA1c,L21:Cholesterol,...,L24:HDL Cholesterol,L2:BUN,L3:Creatinine,L4:Sodium,L5:Potassium,L6:Chloride,L7:Bicarbonate,L8:Uric,L9:Calcium,Systolic BP
0,20.72,1.38,76,3.4,4.4,9.41,313,100,5.3,193,...,65,6.5,0.82,143,3.8,103,29.7,6.5,9.2,138
1,20.27,1.36,66,3.4,4.4,6.43,223,89,5.8,205,...,76,16.7,0.88,140,4.3,102,24.9,6.5,9.2,134
2,19.05,1.33,57,3.4,4.4,5.42,434,88,5.5,198,...,60,14.5,1.10,141,4.4,102,27.2,6.5,9.2,103
3,23.88,1.61,62,3.4,4.4,6.43,223,93,5.3,179,...,68,15.4,0.72,131,4.2,96,28.2,6.5,9.2,128
4,23.88,1.61,70,3.4,4.4,6.43,223,111,6.2,143,...,51,20.8,1.08,140,4.4,102,24.9,6.5,9.2,140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,23.88,1.61,87,3.4,4.4,6.43,223,135,6.6,181,...,48,16.7,2.04,137,4.6,99,27.4,6.5,9.2,147
52615,23.88,1.61,86,4.0,4.2,6.43,223,133,7.2,125,...,47,28.3,1.87,141,5.2,104,23.5,6.5,9.2,143
52616,23.88,1.61,79,3.4,4.4,6.43,223,95,5.8,170,...,51,15.5,1.09,141,4.4,110,19.4,10.9,9.2,131
52617,23.88,1.61,70,3.4,3.2,18.70,298,95,5.8,121,...,51,40.8,2.14,126,4.5,90,22.6,7.3,9.2,138
